In [1]:
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Get data from OpenPrescribing GBG measure equivalent view

In [2]:
sql = """
SELECT
  date(month) as month,
  pct,
  practice,
  SUM(CASE WHEN (possible_savings >= 2
    OR possible_savings <=-2) THEN items ELSE 0 END) AS items,
  SUM(CASE WHEN (possible_savings >= 2
    OR possible_savings <=-2) THEN possible_savings ELSE 0 END) AS possible_savings,
  SUM(net_cost) AS net_cost
FROM
  ebmdatalab.alex.vw__ghost_generic_measure_paper_2019_08
INNER JOIN
  ebmdatalab.hscic.practices
ON
  practice = code
  AND setting = 4
where
month <= "2019-05-01"  ##this is the latest month of data when the data is extracted. 
GROUP BY
  month,
  pct,
  practice
"""
ghost_df = bq.cached_read(sql, csv_path='ghost_generics.zip',use_cache=True)

In [3]:
ghost_df.head()

,month,pct,practice,items,possible_savings,net_cost
0,2014-03-01,03Q,B82026,1065,64.9390,171882.52
1,2014-03-01,03N,C88077,11,6.8506,37022.98
2,2014-03-01,05T,Y03602,1,-2.4335,25838.12
3,2014-03-01,04C,C82614,2,-2.3350,21840.09
4,2014-03-01,07N,G83066,9,-0.5510,58291.17


In [4]:
ghost_df.month.max()

'2019-05-01'

## Practice Level Data

In [5]:
practice = ghost_df.copy()
practice['calc_val'] = (practice['possible_savings']/practice['net_cost'])*100
practice['month'] = pd.to_datetime(practice['month'])
practice = practice.sort_values(['practice','month'])
practice = practice.loc[practice['month']>='2013-05-01'] ##ncso rules were different before this
practice = practice.replace([np.inf, -np.inf], np.nan)
practice.head()

,month,pct,practice,items,possible_savings,net_cost,calc_val
671188,2013-05-01,00K,A81001,3,3.2850,32039.11,0.010253
777714,2013-06-01,00K,A81001,3,-2.1272,27799.60,-0.007652
52515,2013-07-01,00K,A81001,0,0.0000,30349.69,0.000000
153854,2013-08-01,00K,A81001,3,3.3522,30766.65,0.010896
259837,2013-09-01,00K,A81001,1,32.1800,29514.69,0.109030


In [ ]:
practice.nunique()

In [ ]:
## here we describe ranges for Decmebr 2018 - month of surfacing  the issue
df_practice_dec18 = practice.copy().loc[(practice["month"]== "2018-12-01")]
df_practice_dec18.describe()

In [ ]:
df_practice_dec18.nunique()

In [ ]:
df_practice_dec18.possible_savings.quantile(0.6)

In [ ]:
df_practice_dec18.calc_val.quantile(0.99)

## Get Data on EHRs

In [ ]:
epr = pd.read_csv('GPSoC/complete.csv')
epr = epr.rename(index=str, columns={"Principal Supplier": "Principal_Supplier",
                                     "Principal System": "Principal_System"})
epr['Date'] = pd.to_datetime(epr['Date'])
epr.loc[epr['Principal_Supplier']=='TPP','Principal_Supplier'] = 'SystmOne'
epr.loc[epr['Principal_Supplier']=='INPS','Principal_Supplier'] = 'Vision'
epr = epr.drop('Unnamed: 0', axis=1)
epr.head(5)

In [ ]:
numbers = practice[['month','practice', 'possible_savings','net_cost']]
first = epr.loc[epr['Date']=='2016-03-01',['ODS','Principal_Supplier']]
last = epr.loc[epr['Date']=='2018-12-01',['ODS','Principal_Supplier']]
by_epr = numbers.merge(epr, how='left', left_on=['practice','month'], right_on=['ODS','Date'])
by_epr = by_epr.merge(first,
                      how='left',
                      left_on='practice',
                      right_on='ODS',
                      suffixes=('','_first')
                     )
'''by_epr = by_epr.merge(last,
                      how='left',
                      left_on='practice',
                      right_on='ODS',
                      suffixes=('','_last')
                     )'''
by_epr.loc[by_epr['month']<'2016-03-01','Principal_Supplier'] = by_epr.loc[by_epr['month']<'2016-03-01','Principal_Supplier_first']
#by_epr.loc[by_epr['month']>'2018-12-01','Principal_Supplier'] = by_epr.loc[by_epr['month']>'2018-12-01','Principal_Supplier_last']
by_epr = by_epr.drop(columns=['Date',
                              'ODS',
                              'Principal_System',
                              'ODS_first',
                              'Principal_Supplier_first'])#,
                              #'ODS_last',
                              #'Principal_Supplier_last'])
by_epr['Principal_Supplier'] = by_epr['Principal_Supplier'].str.strip()
by_epr2 = by_epr.groupby(['month','Principal_Supplier']).sum()
by_epr2 = by_epr2.unstack()
by_epr2.head(5)

In [ ]:
by_epr.sort_values('month')

In [ ]:
ehr_by_ccg = pd.merge(by_epr, practice[['practice', 'month' ,'calc_val']].drop_duplicates(), left_on=['practice', 'month'], right_on= ['practice', 'month'], how='left')
ehr_by_ccg.head(5)

In [ ]:
df_sysone_prac = ehr_by_ccg.loc[ehr_by_ccg['Principal_Supplier']=='SystmOne']
df_sysone_prac.head(5)

In [ ]:
##here we show ranges for December 2018
df_sysymone_dec18 = df_sysone_prac.copy().loc[(df_sysone_prac["month"]== "2018-12-01")]
df_sysymone_dec18.describe()

In [ ]:
##here we calculate the measure value at the 99th centile for systmone
df_sysymone_dec18.calc_val.quantile(0.99)

In [ ]:
df_emis_prac = ehr_by_ccg.loc[ehr_by_ccg['Principal_Supplier']=='EMIS']




In [ ]:
df_emis_dec18 = df_emis_prac.copy().loc[(df_emis_prac["month"]== "2018-12-01")]
df_emis_dec18.calc_val.quantile(0.99)

# CCG level data

In [ ]:
ccg = ghost_df.copy()
ccg['month'] = pd.to_datetime(ccg['month'])
ccg = ccg.groupby(['pct','month'],as_index=False).sum()
ccg['calc_val'] = (ccg['possible_savings']/ccg['net_cost'])*100
ccg = ccg.sort_values(['pct','month'])
ccg = ccg.loc[ccg['month']>='2013-05-01']
ccg = ccg.replace([np.inf, -np.inf], np.nan)
#ccg = ccg.dropna(axis=0)

ccg.head()

In [ ]:
##this is finding those CCGs who have changed
top_n = ccg.loc[ccg['month']=='2019-01-01']
top_n = top_n.sort_values('calc_val',ascending=False)
top_n = top_n.head(30)
before = top_n[['pct','calc_val']].set_index('pct')
top_n = top_n[['pct']]
top_n = top_n.set_index('pct')

most_recent = ccg.loc[ccg['month']==ccg['month'].max(),['pct','calc_val']]
most_recent = most_recent.merge(top_n,how='right',left_on='pct',right_index=True)
most_recent = most_recent.set_index('pct')

'''
change = before - most_recent
change = change - change.min()
change = change / (change.max()-change.min())
change = change['calc_val']
#change = 1-change
'''
change = before - most_recent
change = change / before
change = change['calc_val']

ccg_unstacked = ccg.loc[ccg['month']>='2016-06-01']
ccg_unstacked = ccg_unstacked.merge(top_n,how='right',on='pct')
ccg_unstacked = ccg_unstacked.groupby(['month','pct']).sum()
ccg_unstacked = ccg_unstacked.unstack()

#from matplotlib import cm
#colors = [cm.coolwarm(x) for x in change]
def color_pick(x):
    if x < 0.25:
        return (0.75,0.75,0.75,0.6)
    else:
        return (0.9,0.3,0.15,0.8)
colors = [color_pick(x) for x in change]

f = plt.figure(figsize=(14, 9))
layout = gridspec.GridSpec(1, 1, figure=f)
ax = plt.subplot(layout[0])
ccg_unstacked['calc_val'].plot(ax=ax,color=colors)
#ax.axvline('2018-12-01',color='r',linestyle=':', label='DataLab Discover and Announce GBG')
#ax.axvline('2019-02-01',color='k',linestyle='--', label='SystmOne deploy fix') ## it was last week of Feb, I think a case for setting as march
ax.text('2017-11-01',
        0.22,
        'CCGs reducing by >25% since January 2019 = {}'.format((change >= 0.25).sum()),
        fontsize=14)
ax.set_ylabel(' % Excess costs as a percentage of total generic spending')
ax.set_ylim(0,)
ax.legend().remove()
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
plt.savefig(r'gbg_images\CCGs_that_have_changed.png')
plt.show()

In [ ]:
## ensuring the format is consistent for pounds and pence
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
## here we calculate ranges for december 2018
ccg_dec18 = ccg.copy().loc[(ccg["month"]== "2018-12-01")]
ccg_dec18.describe()

# Most recent month map


In [ ]:
plt.figure(figsize=(12, 7))
ccg_for_map = ccg.loc[ccg['pct']!='112']
last_month = ccg_for_map.loc[ccg['month']==ccg['month'].max()]
maps.ccg_map(last_month, title=" May 2019: Excess spend as a % of generic spending",
             column='calc_val', 
             separate_london=True)
plt.savefig(r'gbg_images\gbg_fig3_ccgmap.png')
plt.show()

# Declie plots

In [ ]:
f = plt.figure(figsize=(14, 5))
#f.suptitle('Sharing Y axis')
f.autofmt_xdate()
layout = gridspec.GridSpec(1, 2, figure=f)
left_plot = plt.subplot(layout[0])
right_plot = plt.subplot(layout[1], sharey=left_plot)  # Share the Y axis 

# ...and because it's shared, suppress ticks on the second chart
plt.setp(right_plot.get_yticklabels(), visible=False)

charts.deciles_chart(
    ccg,
    period_column='month',
    column='calc_val',
    title=" (a) CCG variation",
    ylabel="Excess spend as a\nproportion of generic spending (%) ",
    show_outer_percentiles=True,
    show_legend=False,
    ax=left_plot,  
) 


charts.deciles_chart(
    practice,
    period_column='month',
    column='calc_val',
    title=" (b) Practice variation",
    ylabel="",
    show_outer_percentiles=True,
    show_legend=False,
    ax=right_plot,
) 
plt.ylim(0, 5)

charts.deciles_chart(
        df_sysone_prac,
        period_column='month',
        column='calc_val',
        title=" (c) Practice variation where\nSystmOne is EHR",
        ylabel= "Excess spend as a\nproportion of generic spending (%)",
        show_outer_percentiles=True,
        show_legend=True,
   
)
plt.ylim(0, 5)
plt.savefig('deciles.png', format='png', dpi=300,bbox_inches='tight')
plt.savefig(r'gbg_images\gbg_fig2_deciles_ccgprac.png')
plt.show()

# Total items and total excess costs

In [ ]:
totals = practice.groupby('month').sum()

In [ ]:
f = plt.figure(figsize=(8, 8))
layout = gridspec.GridSpec(2, 1, figure=f)
top_plot = plt.subplot(layout[0])
bottom_plot = plt.subplot(layout[1], sharex=top_plot)  # Share the Y axis 

ax = totals['possible_savings'].plot(ax=top_plot)
ax.set_ylabel('total excess costs (£)')
ax.set_ylim(0,)
ax.set_title('Figure 1a Excess Costs')
ax = totals['items'].plot(ax=bottom_plot)
ax.set_ylabel('total items')
ax.set_ylim(0,)
ax.set_title('Figure 1b Total Items')
plt.savefig('Fig 1.png', format='png', dpi=300,bbox_inches='tight')
plt.savefig(r'gbg_images\gbg_fig1_totalexcess_costitems.png')
plt.show()

## Annual totals
Ignore 2013 and 2019, as they're not complete years

In [ ]:
totals[['items','possible_savings','net_cost']].resample('Y').sum()

# Stratify by EHR

Rerun GPSoC file aggregation if new months need to be added:

In [ ]:
#%run -i "GPSoC/make_csv.py"

In [ ]:
sql = """
        SELECT
          DATE(Date) AS Date,
          ODS,
          TRIM(Principal_Supplier) AS Principal_Supplier,
          TRIM(Principal_System) AS Principal_System
        FROM
          alex.vendors software"""
epr = bq.cached_read(sql, csv_path='software_vendor.zip')
epr['Date'] = pd.to_datetime(epr['Date'])
epr.loc[epr['Principal_Supplier']=='TPP','Principal_Supplier'] = 'SystmOne'
epr.loc[epr['Principal_Supplier']=='INPS','Principal_Supplier'] = 'Vision'
epr.head()

In [ ]:
epr_march19 = epr.loc[epr['Date']=='2019-03-01',['ODS','Principal_Supplier', 'Date']]
epr_march19.head(5)

In [ ]:
epr_march19.groupby('Principal_Supplier').count()

In [ ]:
##here we work out practice use of EHR in december for cohort characteristics
epr_dec19 = epr.loc[epr['Date']=='2018-12-01',['ODS','Principal_Supplier', 'Date']]
epr_dec19.head(5)

In [ ]:
epr_dec19.groupby('Principal_Supplier').count()

In [ ]:
numbers = practice[['month','practice', 'possible_savings','net_cost']]
first = epr.loc[epr['Date']=='2016-03-01',['ODS','Principal_Supplier']]
last = epr.loc[epr['Date']=='2018-12-01',['ODS','Principal_Supplier']]
by_epr = numbers.merge(epr, how='left', left_on=['practice','month'], right_on=['ODS','Date'])
by_epr = by_epr.merge(first,
                      how='left',
                      left_on='practice',
                      right_on='ODS',
                      suffixes=('','_first')
                     )
'''by_epr = by_epr.merge(last,
                      how='left',
                      left_on='practice',
                      right_on='ODS',
                      suffixes=('','_last')
                     )'''
by_epr.loc[by_epr['month']<'2016-03-01','Principal_Supplier'] = by_epr.loc[by_epr['month']<'2016-03-01','Principal_Supplier_first']
#by_epr.loc[by_epr['month']>'2018-12-01','Principal_Supplier'] = by_epr.loc[by_epr['month']>'2018-12-01','Principal_Supplier_last']
by_epr = by_epr.drop(columns=['Date',
                              'ODS',
                              'Principal_System',
                              'ODS_first',
                              'Principal_Supplier_first'])#,
                              #'ODS_last',
                              #'Principal_Supplier_last'])
by_epr['Principal_Supplier'] = by_epr['Principal_Supplier'].str.strip()
by_epr2 = by_epr.groupby(['month','Principal_Supplier']).sum()
by_epr2 = by_epr2.unstack()
by_epr2.head()

In [ ]:
by_epr.head(5)

### EHR Usage by CCG - SystmOne Deciles

In [ ]:
ehr_by_ccg = pd.merge(by_epr, practice[['practice', 'month' ,'calc_val']].drop_duplicates(), left_on=['practice', 'month'], right_on= ['practice', 'month'], how='left')
ehr_by_ccg.head(5)

In [ ]:
df_sysone_prac = ehr_by_ccg.loc[ehr_by_ccg['Principal_Supplier']=='SystmOne']
df_sysone_prac.head(5)

In [ ]:
charts.deciles_chart(
        df_sysone_prac,
        period_column='month',
        column='calc_val',
        title="SystmOne Deciles",
        ylabel="Excess spend as a proportion of generic spending (%) ",
        show_outer_percentiles=True,
        show_legend=True
)
plt.savefig('deciles.png', format='png', dpi=300,bbox_inches='tight')
plt.savefig(r'gbg_images\ehrshare_latestmonth.png')
plt.show()

## Total possible savings

In [ ]:
f = plt.figure(figsize=(10, 6))
layout = gridspec.GridSpec(1, 1, figure=f)
ax = plt.subplot(layout[0])
ax = by_epr2['possible_savings'].plot(ax=ax)
ax.axvline('2018-12-01',color='r',linestyle=':', label='DataLab Identify GBG')
ax.axvline('2019-02-01',color='k',linestyle='--', label='SystmOne deploy fix')
ax.set_title('Figure 1c Total excess cost (£) of Ghost Branded Generic Prescribing by general practices \n in England, grouped by EHR system. May 2013 - May 2019')
ax.legend()
plt.legend(fontsize=14)
ax.set_ylabel('total excess costs (£) ')
ax.set_ylim(0,)
plt.savefig('savings_by_EHR.png', format='png', dpi=300,bbox_inches='tight')
plt.savefig(r'gbg_images\gbg_fig4_totalcosts_ehr.png')
plt.show()

## Total savings/total generic prescribing costs

In [ ]:
measure = by_epr2['possible_savings'] / by_epr2['net_cost']
measure.head()

In [ ]:
measure.plot()
plt.savefig(r'gbg_images\gbg_figx_measures.png')

# SystmOne Only

In [ ]:
systmone = by_epr2['possible_savings']['SystmOne']
systmone.head()

In [ ]:
f = plt.figure(figsize=(8, 5))
layout = gridspec.GridSpec(1, 1, figure=f)
ax = plt.subplot(layout[0])
ax = systmone.plot(ax=ax)
ax.axvline('2018-12-01',color='r',linestyle=':', label='DataLab Identify GBG')
ax.axvline('2019-02-01',color='k',linestyle='--', label='SystmOne delpoy fix') ## it was last week of Feb, I think a case for setting as march
ax.set_ylabel('total excess costs')
ax.set_ylim(0,)
ax.legend()
plt.savefig('systmone_only.png', format='png', dpi=300,bbox_inches='tight')
plt.savefig(r'gbg_images\gbg_fig5_systmone.png')
plt.show()

# Top ten GBG chemicals in 2018

In [ ]:
sql = """
WITH
  chem_map AS(
  SELECT
    DISTINCT chemical_code,
    chemical
  FROM
    ebmdatalab.hscic.bnf)
SELECT
  SUBSTR(bnf_code,1,9) AS chem_code,
  chemical,
  SUM(CASE WHEN (possible_savings >= 2
    OR possible_savings <=-2) THEN items ELSE 0 END) AS items,
  SUM(CASE WHEN (possible_savings >= 2
    OR possible_savings <=-2) THEN possible_savings ELSE 0 END) AS excess_spend
FROM
  ebmdatalab.alex.vw__ghost_generic_measure_paper_2019_08
INNER JOIN
  ebmdatalab.hscic.practices
ON
  practice = code
  AND setting = 4
LEFT JOIN
  chem_map
ON
  SUBSTR(bnf_code,1,9) = chemical_code
WHERE
  month >= '2018-01-01'
  AND month <= '2018-12-01'
GROUP BY
  chem_code,
  chemical
ORDER BY
  excess_spend DESC
"""
by_chem = bq.cached_read(sql, csv_path='by_chem.zip')

In [ ]:
by_chem.head(10)

In [ ]:
by_chem.sort_values('items',ascending=False).head(10)

In [ ]:
plt = maps.ccg_map(
    last_month, 
    title="CCG list sizes", 
    column='calc_val', 
    cartogram=True, 
    separate_london=False)
plt.show()